In [1]:
import numpy
import librosa
import matplotlib.pyplot as plot
import sklearn
from sklearn.model_selection import train_test_split
import random
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.utils import to_categorical
import pandas as pd
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn import tree
import graphviz 
%matplotlib inline

batch_size = 2
epochs = 1000
clip_length_seconds = 120
downsampling_rate = 25
split_ratio = 0.8
random_state = random.randint(1,101)
print("Random state: " + str(random_state))
count_files = 10
y = numpy.flip(numpy.array([1, 2, 1, 2, 2, 0, 0, 2, 1, 0, 0, 2, 2, 2]))

y = y[:count_files,]

X_vectors = []
y_vectors = []

for i in range(1, count_files + 1):
    #filename = 'vid' + str(i) + '.wav'
    filename = 'download.wav'
    
    wave, sr = librosa.load(filename, mono=True, sr=None)
    print(sr)
    print(wave.shape)
    #print('Sample rate: ' + str(sr))    
    #print('Wave shape for i = ' + str(i) + ': ' + str(wave.shape))
    
    for j in range(0, wave.shape[0] / clip_length_seconds / sr):
        start = j * clip_length_seconds * sr
        end = (j + 1) * clip_length_seconds * sr
        
        wave_clip = wave[start:end]
        #print('Clip shape for j = ' + str(j) + ': ' + str(wave_clip.shape))

        wave_downsampled = wave_clip[::downsampling_rate]
        #print('Downsample shape for j = ' + str(j) + ': ' + str(wave_downsampled.shape))

        mfcc = librosa.feature.mfcc(wave_downsampled, sr)
        #print('Padded mfcc shape for j = ' + str(j) + ': ' + str(mfcc.shape))
    
        X_vectors.append(mfcc)
        y_vectors.append(y[i - 1])
    
        #plot.subplot(312)
        #mfcc -= (numpy.mean(mfcc,axis=0) + 1e-8)
        #plot.imshow(mfcc.T, cmap=plot.cm.jet, aspect='auto')
        # plot.xticks(numpy.arange(0, (mfcc.T).shape[1], int((mfcc.T).shape[1] / 6)), ['0s', '0.5s', '1s', '1.5s','2.5s','3s','3.5'])
        #ax = plot.gca()
        #ax.invert_yaxis()
        #plot.title('the Normalized MFCC spectrum image for video ' + str(i))
        #plot.show()
    
X = numpy.array(X_vectors)
y = numpy.array(y_vectors)

assert X.shape[0] == y.shape[0]
totaccsvn = 0
totaccdt = 0
for num in range(0,20):
    random_state = random.randint(1,101)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = (1 - split_ratio), random_state = random_state, shuffle = True)
    # Support vector machine (baseline)
    #print(X_train[0])
    X_train_t = [i.flatten() for i in X_train]
    X_test_t = [i.flatten() for i in X_test]
    svclassifier = SVC(kernel='linear')  
    svclassifier.fit(X_train_t, y_train)
    y_pred = svclassifier.predict(X_test_t)  
    accuracysvn = accuracy_score(y_test,y_pred)
    #print("The svn accuracy is:" + str(accuracy))
    totaccsvn += accuracysvn
    #print(confusion_matrix(y_test,y_pred))  
    #print(classification_report(y_test,y_pred))

# (Basic) convolutional neural network
    
    accuracydt = 0
    clf = tree.DecisionTreeClassifier()
    clf = clf.fit(X_train_t, y_train)
    y_pred = clf.predict(X_test_t)
    accuracydt = accuracy_score(y_test,y_pred)
    totaccdt += accuracydt
    #dot_data = tree.export_graphviz(clf, out_file=None) 
    #graph = graphviz.Source(dot_data) 
    #graph.render("Booooring!!!!!") 
    #print("The decision tree accuracy is:" + str(accuracy))
print("The average accuracy of SVN:" + str(totaccsvn/20.0))
print("The average accuracy of DTree:" + str(totaccdt/20.0))


#dot_data = tree.export_graphviz(clf, out_file=None) 
#graph = graphviz.Source(dot_data) 
#graph.render("Booooring!!!!!")
#graph = graphviz.Source(dot_data)  
#graph 

"""

X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], X_train.shape[2], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], X_test.shape[2], 1)

y_train_hot = to_categorical(y_train)
y_test_hot = to_categorical(y_test)

model = Sequential()
model.add(Conv2D(64, kernel_size=(5, 5), activation='relu', input_shape=(X_train.shape[1], X_train.shape[2], X_train.shape[3])))
model.add(MaxPooling2D(pool_size=(5, 5)))
#model.add(Dropout(rate = 1 - 0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
#model.add(Dropout(rate = 1 - 0.25))
model.add(Dense(3, activation='softmax'))
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])
model.fit(X_train, y_train_hot, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(X_test, y_test_hot))
"""

Using TensorFlow backend.


Random state: 4
8000
(19360,)
8000
(19360,)
8000
(19360,)
8000
(19360,)
8000
(19360,)
8000
(19360,)
8000
(19360,)
8000
(19360,)
8000
(19360,)
8000
(19360,)


ValueError: Expected 2D array, got 1D array instead:
array=[].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.